In [2]:
# here i will try implrmrnting LSTM model #
# copied from https://analyticsindiamag.com/how-to-do-multivariate-time-series-forecasting-using-lstm/


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf 

In [9]:
data = pd.read_csv('../input/metro-interstate-traffic-volume/Metro_Interstate_Traffic_Volume.csv')
print(data.shape)
data.head()

(48204, 9)


,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918


In [8]:
for i in data.select_dtypes('object').columns:
    le = LabelEncoder().fit(data[i])
    data[i] = le.transform(data[i]) 
X_scaler = MinMaxScaler()
Y_scaler = MinMaxScaler()
X_data = X_scaler.fit_transform(data[['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main',
    'weather_description','traffic_volume']])
Y_data = Y_scaler.fit_transform(data[['traffic_volume']])
X_data.shape

(48204, 8)

array([[0.63636364, 0.92972555, 0.        , ..., 0.1       , 0.64864865,
        0.76167582],
       [0.63636364, 0.93320863, 0.        , ..., 0.1       , 0.05405405,
        0.62032967],
       [0.63636364, 0.93391815, 0.        , ..., 0.1       , 0.51351351,
        0.65480769],
       ...,
       [0.63636364, 0.91182636, 0.        , ..., 1.        , 0.56756757,
        0.29656593],
       [0.63636364, 0.90976231, 0.        , ..., 0.1       , 0.51351351,
        0.19917582],
       [0.63636364, 0.90985906, 0.        , ..., 0.1       , 0.51351351,
        0.13104396]])